In [1]:
import random
import numpy
import tensorflow
import keras

from keras.preprocessing.image           import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

In [2]:
main_dir       = "BrainTumorDataSet"
data_generator = ImageDataGenerator(validation_split = 0.2)

image_width  = 128
image_height = 128

train_dataset = data_generator.flow_from_directory(main_dir, 
                                                   target_size = (image_width, image_height),
                                                   batch_size = 256,
                                                   class_mode = "binary",
                                                   color_mode = "rgb",
                                                   shuffle = True,
                                                   seed = 57,
                                                   subset = "training")

test_dataset = data_generator.flow_from_directory(main_dir,
                                                  target_size = (image_width, image_height),
                                                  batch_size = 256,
                                                  class_mode = "binary",
                                                  color_mode = "rgb",
                                                  shuffle = True,
                                                  seed = 57,
                                                  subset = "validation")

Found 3283 images belonging to 2 classes.
Found 819 images belonging to 2 classes.


In [3]:
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (image_height, image_width, 3))
base_model.trainable = False

model = keras.Sequential()

model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(rate = 0.3))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dropout (Dropout)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 1)                 8193      
                                                                 
Total params: 14,722,881
Trainable params: 8,193
Non-trainable params: 14,714,688
_________________________________________________________________


In [4]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
history = model.fit(train_dataset, epochs = 10)

Epoch 1/10
13/13 [==============================] - 166s 13s/step - loss: 2.8351 - accuracy: 0.7201
Epoch 2/10
13/13 [==============================] - 201s 15s/step - loss: 1.2731 - accuracy: 0.8492
Epoch 3/10
13/13 [==============================] - 191s 15s/step - loss: 0.7586 - accuracy: 0.8983
Epoch 4/10
13/13 [==============================] - 209s 16s/step - loss: 0.5538 - accuracy: 0.9126
Epoch 5/10
13/13 [==============================] - 230s 18s/step - loss: 0.4069 - accuracy: 0.9235
Epoch 6/10
13/13 [==============================] - 233s 18s/step - loss: 0.3842 - accuracy: 0.9360
Epoch 7/10
13/13 [==============================] - 286s 22s/step - loss: 0.3261 - accuracy: 0.9443
Epoch 8/10
13/13 [==============================] - 289s 22s/step - loss: 0.2558 - accuracy: 0.9519
Epoch 9/10
13/13 [==============================] - 289s 22s/step - loss: 0.1952 - accuracy: 0.9583
Epoch 10/10
13/13 [==============================] - 273s 21s/step - loss: 0.1621 - accuracy: 0.9625

In [5]:
model.save('TransferLearningCancerDetectionModel.h5')

In [6]:
score = model.evaluate(test_dataset)
print('Accuaracy on test dataset: ', score[1])

4/4 [==============================] - 58s 13s/step - loss: 0.2465 - accuracy: 0.9609
Accuaracy on test dataset:  0.9609279632568359
